<a href="https://colab.research.google.com/github/FlyAIBox/langchain-academy/blob/fly101/module-1/deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 部署（Deployment）

## 回顾（Review）

我们已经构建了一个带有“记忆能力”的智能体（agent）：

- `act`：让大模型调用预先注册的工具（Tools）
- `observe`：将工具的输出结果反馈给大模型
- `reason`：让大模型基于工具输出进行推理，决定下一步动作（继续调用工具，或直接回复）
- `persist state`：使用内存检查点（in-memory checkpointer）保存对话与状态，支持长对话与中断续跑

## 目标（Goals）

下面我们将介绍如何在本地通过 Studio，以及在 `LangGraph Cloud` 上，实际部署并运行这个智能体。

In [1]:
%%capture --no-stderr
# %pip install --quiet -U langgraph_sdk langchain_core
%pip install --quiet langgraph_sdk==0.2.6 langchain_core==0.3.75

## 核心概念（Concepts）

需要先理解以下核心概念：

`LangGraph` ——
- 一个用于构建智能体（agent）工作流的 Python / JavaScript 库。

`LangGraph API` ——
- 将 graph（工作流）代码打包成可服务化的接口。
- 提供任务队列，用于管理异步执行。
- 提供状态持久化，保证多轮交互的上下文连续。

`LangGraph Cloud` ——
- 官方托管的 LangGraph API 云服务。
- 支持从 GitHub 仓库直接部署 graph。
- 提供监控与追踪能力（monitoring & tracing）。
- 每次部署都会获得唯一访问 URL。

`LangGraph Studio` ——
- LangGraph 应用的集成开发环境（IDE）。
- 以前后端分离方式工作：前端是可视化界面，后端依赖 API。
- 可在本地运行，也可连接云端部署进行调试。

`LangGraph SDK` ——
- 用于以编程方式访问 LangGraph 的 Python 库。
- 无论连接本地还是云端，接口保持一致。
- 支持创建客户端、管理 assistant、管理 thread、执行 run 等。

## 本地测试（Testing Locally）

**⚠️ 说明**

我们已更新 Studio，使其可在本地运行并通过浏览器访问。这是当前推荐的运行方式（替代视频中展示的桌面应用）。本地开发服务器的文档见：[本地开发服务器概念](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/#local-development-server) 与 [如何运行本地 Studio](https://langchain-ai.github.io/langgraph/how-tos/local-studio/#run-the-development-server)。

在本模块的 `/studio` 目录下，运行以下命令启动本地开发服务器：

```
langgraph dev
```

启动后，你应看到类似输出：
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

然后在浏览器打开 Studio 界面：`https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024`。

In [ ]:
if 'google.colab' in str(get_ipython()):
    raise Exception("Unfortunately LangGraph Studio is currently not supported on Google Colab")

In [ ]:
from langgraph_sdk import get_client

In [ ]:
# 本地开发服务器的 URL
URL = "http://127.0.0.1:2024"
client = get_client(url=URL)

# 查询当前服务中可用的所有图（assistants）
assistants = await client.assistants.search()

In [ ]:
assistants[-3]

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'config': {},
 'metadata': {'created_by': 'system'},
 'name': 'agent',
 'created_at': '2025-03-04T22:57:28.424565+00:00',
 'updated_at': '2025-03-04T22:57:28.424565+00:00',
 'version': 1}

In [ ]:
# 创建线程（thread）以跟踪一次运行（run）的状态
thread = await client.threads.create()

现在我们可以使用 [`client.runs.stream`](https://langchain-ai.github.io/langgraph/concepts/low_level/#stream-and-astream) 来运行智能体，需要提供：

- `thread_id`：线程标识，用于关联本次运行的状态
- `graph_id`：要运行的图（即智能体）的标识
- `input`：输入数据（例如消息）
- `stream_mode`：流式返回的模式

关于流式处理（streaming）我们会在后续模块详细展开。这里先理解：当设置 `stream_mode="values"` 时，我们会在图的每一步后，[流式](https://langchain-ai.github.io/langgraph/cloud/how-tos/stream_values/)返回“完整状态值”。

当前步的状态存放在返回块 `chunk.data` 中。

In [ ]:
from langchain_core.messages import HumanMessage

# 构造输入
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# 以流式方式运行并打印每一步最新一条消息
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'cdbd7bd8-c476-4ad4-8ab7-4ad9e3654267', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_iIPryzZZxRtXozwwhVtFObNO', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5'}, 'type': 'ai', 'name': None, 'id': 'run-06c7243c-426d-4c81-a113-f1335dda5fb2', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_iIPryzZZxRtXozwwhVtFObNO', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '988cb170-f6e6-43c1-82fd-309f519abe6d', 'tool_call_id': 'c

## 云端测试（Testing with Cloud）

可以通过 LangSmith 将 graph 部署到云端，参考官方指南：[从 GitHub 部署到 LangGraph Cloud](https://langchain-ai.github.io/langgraph/cloud/quick_start/#deploy-from-github-with-langgraph-cloud)。

### 在 GitHub 创建新仓库

- 进入你的 GitHub 账户
- 点击右上角“+”并选择“New repository”
- 命名仓库（例如：`langchain-academy`）

### 将本地项目关联到 GitHub 远程仓库

- 回到你本地克隆本课程代码的终端
- 将新建的 GitHub 仓库添加为远程：

```
git remote add origin https://github.com/your-username/your-repo-name.git
```
- 推送代码：
```
git push -u origin main
```

### 在 LangSmith 连接你的 GitHub 仓库

- 打开 [LangSmith](https://smith.langchain.com/)
- 在左侧面板点击 `deployments`
- 点击 `+ New Deployment`
- 选择你刚创建的 GitHub 仓库（如 `langchain-academy`）
- 将 `LangGraph API config file` 指向对应模块的 `studio` 目录
- 例如 module-1：`module-1/studio/langgraph.json`
- 设置所需的 API 密钥（可从 `module-1/studio/.env` 复制）

![Deployment step 2](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fd61c93d48e5d0f47_deployment2.png)

### 使用你的部署

可以通过多种方式与云端部署交互：

- 通过 [SDK](https://langchain-ai.github.io/langgraph/cloud/quick_start/#use-with-the-sdk)
- 通过 [LangGraph Studio](https://langchain-ai.github.io/langgraph/cloud/quick_start/#interact-with-your-deployment-via-langgraph-studio)

![Deployment step 3](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fa159a09a51d601de_deployment3.png)

如果在本 Notebook 中使用 SDK，请确保已设置环境变量 `LANGSMITH_API_KEY`。

In [ ]:
import os, getpass

def _set_env(var: str):
    # 若未设置，则通过交互方式安全读取并设置环境变量
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGSMITH_API_KEY")

In [ ]:
# 将此处替换为你在云端部署的 graph URL
URL = "https://langchain-academy-8011c561878d50b1883f7ed11b32d720.default.us.langgraph.app"
client = get_client(url=URL)

# 查询云端部署中托管的所有图（assistants）
assistants = await client.assistants.search()

In [ ]:
# 选择一个待运行的智能体（assistant）
agent = assistants[0]

In [ ]:
agent

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'created_at': '2024-08-23T17:58:02.722920+00:00',
 'updated_at': '2024-08-23T17:58:02.722920+00:00',
 'config': {},
 'metadata': {'created_by': 'system'}}

In [ ]:
from langchain_core.messages import HumanMessage

# 创建线程（thread）以跟踪一次运行（run）的状态
thread = await client.threads.create()

# 构造输入
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# 以流式方式运行并打印每一步最新一条消息
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8ea04559-f7d4-4c82-89d9-c60fb0502f21', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27'}, 'type': 'ai', 'name': None, 'id': 'run-b0ea5ddd-e9ba-4242-bb8c-80eb52466c76', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '1bf558e7-79ef-4f21-bb66-acafbd04677a', 'tool_call_id': 'c